In [1]:
from src.data import _load_readmission_dataset
from src.helper_functions import _prepare_rt_tensors
from src.models import *
from src.losses import *
from src.training import *
from src.metrics import *

In [2]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
x, t, e, d = _load_readmission_dataset(sequential=True)

/home/ayush/DeepReEvent/src/data.py:44: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data['max_time'] = data.groupby('id')['t.stop'].transform(max)


In [4]:
readmission_tensor = _prepare_rt_tensors(x, t, e, d)
locals().update(readmission_tensor) # create variables from dictionary

/home/ayush/DeepReEvent/src/helper_functions.py:83: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  x = torch.tensor(x, dtype=torch.float32)


In [5]:
# model and training parameters

input_size = len(x[0]) 
output_size = int(max(t))
hidden_size = 8   # Number of units in the RNN layer

model = SimpleRNN(input_size, hidden_size, output_size, 2)

# Instantiate the model

num_epochs = 1000
patience = 3  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
wait = 0
loss_function = recurrent_terminal_loss

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-7, lr=1e-4)

In [6]:
#prepare data loaders

train_dataset = torch.utils.data.TensorDataset(x_train, t_train, e_train, d_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)

val_dataset = torch.utils.data.TensorDataset(x_val, t_val, e_val, d_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

In [7]:
train_validate_rt_model(model, train_dataloader, val_dataloader, loss_function, optimizer, num_epochs, patience, print_every=10)

Epoch: 0   Training loss: 15.5889   Validation loss: 14.3970


Epoch: 10   Training loss: 15.0973   Validation loss: 13.9766
Epoch: 20   Training loss: 14.6393   Validation loss: 13.5861
Epoch: 30   Training loss: 14.1956   Validation loss: 13.2081
Epoch: 40   Training loss: 13.7404   Validation loss: 12.8193
Epoch: 50   Training loss: 13.2408   Validation loss: 12.3922
Epoch: 60   Training loss: 12.6737   Validation loss: 11.9105
Epoch: 70   Training loss: 12.0476   Validation loss: 11.3870
Epoch: 80   Training loss: 11.4095   Validation loss: 10.8632
Epoch: 90   Training loss: 10.8130   Validation loss: 10.3814
Epoch: 100   Training loss: 10.2845   Validation loss: 9.9597
Epoch: 110   Training loss: 9.8263   Validation loss: 9.5971
Epoch: 120   Training loss: 9.4302   Validation loss: 9.2854
Epoch: 130   Training loss: 9.0864   Validation loss: 9.0159
Epoch: 140   Training loss: 8.7861   Validation loss: 8.7808
Epoch: 150   Training loss: 8.5219   Validation loss: 8.5740
Epoch: 160   Training loss: 8.2880   Validation loss: 8.3908
Epoch: 170   T

In [8]:
test_predictions = model(x_test).squeeze(-1)

survival_predictions = test_predictions[:, :, 0:1].squeeze(-1)
recurrent_predictions = test_predictions[:, :, 1:2].squeeze(-1)

In [9]:
calculate_survival_metrics(survival_predictions, d_train, t_train, d_test, t_test, d_val, t_val)

{'Brier Score': array([0.01711155, 0.04105541, 0.04318445, 0.03670069, 0.03524066,
        0.03828672, 0.03856495, 0.03955116, 0.04226858, 0.04874788,
        0.04113356, 0.03538256, 0.03148225, 0.03431167, 0.01815343]),
 '25th Quantile CI': 0.9590696446501267,
 '50th Quantile CI': 0.9590696446501267,
 '75th Quantile CI': 0.9567284019000687}

In [18]:
h = survival_predictions[d_test == 1][0]
t = t_test[d_test == 1][0]
print(h)

tensor([0.0943, 0.0763, 0.0510, 0.0371, 0.0475, 0.0285, 0.0454, 0.0412, 0.0254,
        0.0349, 0.0634, 0.0375, 0.0246, 0.0855, 0.0622, 0.0499, 0.0582, 0.0855,
        0.0490, 0.0982, 0.0731, 0.6544], grad_fn=<SelectBackward0>)


In [19]:
h[1]

tensor(0.0763, grad_fn=<SelectBackward0>)

In [12]:
survival_predictions

h = survival_predictions[0]

L1 = torch.log(h[t])
# Log-likelihood contribution from survival until time t
L2 = torch.sum(torch.log(1 - h[0:t]))

## Experimenting

In [13]:
out_risk = recurrent_predictions

current_time = 20

expected_number_of_events = torch.zeros(out_risk.size(0))

for i in range(out_risk.size(0)):
    clamped_time = min(current_time, t_test[i].item())
    expected_number_of_events[i] = out_risk[i, 0:clamped_time].sum()

print(expected_number_of_events)


tensor([2.2043, 1.0303, 1.4937, 2.7117, 1.2854, 2.7399, 2.2151, 2.9163, 0.4369,
        1.2414, 0.0000, 2.6816, 2.2043, 1.8762, 3.0435, 0.8154, 0.4162, 2.9163,
        1.8798, 2.7399, 2.2940, 1.7662, 1.6680, 1.8619, 2.4202, 2.7054, 2.7056,
        1.0773, 0.6896, 2.7054, 1.8762, 0.6963, 0.3369, 2.1819, 0.2279, 2.7054,
        1.8838, 0.2336, 1.0773, 1.8798, 1.6719, 1.3532, 0.5149, 0.8803, 1.6680,
        2.1825, 0.0000, 1.6667, 0.5508, 1.6666, 1.0120, 1.9251, 0.6909, 1.6667,
        1.6028, 2.1819, 0.6909, 1.4815, 1.2379, 2.2940, 1.2379, 0.0000, 0.4796,
        1.4872, 1.6489, 1.6481, 0.6748, 1.6666, 1.8798, 2.3226, 1.9578, 0.4008,
        2.2679, 1.0022, 1.6667, 0.5353, 2.3313, 1.3532, 1.3507, 1.6481, 1.0013],
       grad_fn=<CopySlices>)


In [14]:
mask = e_test < current_time
observed_number_of_events = mask.sum(dim=1)

print(observed_number_of_events[4])
print(expected_number_of_events[4])

print(e_test[4])

out_risk[4][0:8]

tensor(5)
tensor(1.2854, grad_fn=<SelectBackward0>)
tensor([  0,   2,   4,   8,   8, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100])


tensor([0.2335, 0.1828, 0.1396, 0.1481, 0.1433, 0.1948, 0.0897, 0.1535],
       grad_fn=<SliceBackward0>)

In [15]:
h = out_risk[2]
e = e_test[2]
t = t_test[2]

mask = torch.ones_like(h[0:t], dtype=torch.bool)
# Set mask to False for time points where events occurred
mask[e[e < t]] = False
print(mask)
# Calculate the negative log-likelihood for both the event occurrences and non-occurrences
-1 * (torch.sum(torch.log(h[0:t][e[e < t]])) + torch.sum(torch.log(1 - h[0:t][mask])))


tensor([ True,  True, False,  True, False, False,  True])


tensor(5.7868, grad_fn=<MulBackward0>)

In [16]:
print("h", h)
print("e, t", e, t)
print(h[0:t][e[e < t]])

h tensor([0.2882, 0.2267, 0.2029, 0.2196, 0.1678, 0.2449, 0.1436, 0.2011, 0.1949,
        0.1974, 0.2442, 0.2740, 0.3642, 0.1883, 0.3826, 0.2654, 0.4442, 0.5562,
        0.5358, 0.5976, 0.6642, 0.8765], grad_fn=<SelectBackward0>)
e, t tensor([  2,   4,   5,   7, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100]) tensor(7)
tensor([0.2029, 0.1678, 0.2449], grad_fn=<IndexBackward0>)


In [17]:
import pandas as pd
data = pd.read_csv('data/readmission.csv', delimiter=';')
torch.tensor(0.0)

tensor(0.)